# Demand Forecasting Model with Snowpark ML

This notebook trains an XGBoost regression model to predict **units sold** (demand) based on:
- Date features (month, day of week, year)
- Region
- Product category

The model is registered in Snowflake's Model Registry and can be called as a tool from the Snowflake Intelligence Agent to answer questions like:
- "How many units of Electronics will we sell in the West region next month?"
- "What's the forecasted demand for Fitness Wear in the East in Q4?"

## 1. Setup and Imports

In [ ]:
# Snowpark and ML imports
from snowflake.snowpark import Session
from snowflake.snowpark.functions import col, month, dayofweek, year
from snowflake.ml.modeling.xgboost import XGBRegressor
from snowflake.ml.modeling.preprocessing import OrdinalEncoder
from snowflake.ml.registry import Registry
import pandas as pd
import numpy as np

# For evaluation
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [ ]:
# Get active Snowpark session (when running in Snowsight)
from snowflake.snowpark.context import get_active_session
session = get_active_session()

# IMPORTANT: Update these values based on your lab path
# For Manual path (si_*): DATABASE = 'SI_DB', MODEL_NAME = 'SI_DEMAND_FORECAST'
# For Cortex Code path (coco_*): DATABASE = 'COCO_DB', MODEL_NAME = 'COCO_DEMAND_FORECAST'

DATABASE = 'SI_DB'  # Change to 'COCO_DB' for Cortex Code path
SCHEMA = 'RETAIL'
MODEL_NAME = 'SI_DEMAND_FORECAST'  # Change to 'COCO_DEMAND_FORECAST' for Cortex Code path

# Set database and schema context - REQUIRED for temp table operations
session.use_database(DATABASE)
session.use_schema(SCHEMA)

print(f"Using database: {DATABASE}")
print(f"Using schema: {SCHEMA}")
print(f"Model will be registered as: {MODEL_NAME}")

## 2. Load and Prepare Data

In [ ]:
# Load sales data with product info
sales_df = session.table(f"{DATABASE}.{SCHEMA}.SALES")
products_df = session.table(f"{DATABASE}.{SCHEMA}.PRODUCTS")

# Join sales with products to get category
df = sales_df.join(
    products_df,
    sales_df["PRODUCT_ID"] == products_df["PRODUCT_ID"],
    "left"
).select(
    sales_df["DATE"],
    sales_df["REGION"],
    products_df["CATEGORY"],
    sales_df["UNITS_SOLD"]
)

print(f"Total records: {df.count()}")
df.show(5)

In [ ]:
# Feature engineering: extract date features
df_features = df.with_column("MONTH", month(col("DATE"))) \
               .with_column("DAY_OF_WEEK", dayofweek(col("DATE"))) \
               .with_column("YEAR", year(col("DATE")))

# Select features for modeling - UNITS_SOLD is now the TARGET, not a feature
df_model = df_features.select(
    "REGION",
    "CATEGORY", 
    "MONTH",
    "DAY_OF_WEEK",
    "YEAR",
    "UNITS_SOLD"  # This is our target variable
).dropna()

print(f"Records after feature engineering: {df_model.count()}")
df_model.show(5)

## 3. Encode Categorical Features

In [ ]:
# Encode categorical variables (REGION, CATEGORY)
categorical_cols = ["REGION", "CATEGORY"]
output_cols = ["REGION_ENCODED", "CATEGORY_ENCODED"]

encoder = OrdinalEncoder(
    input_cols=categorical_cols,
    output_cols=output_cols
)

encoder.fit(df_model)
df_encoded = encoder.transform(df_model)

df_encoded.show(5)

In [ ]:
# Show the encoding mapping for reference
print("Encoding mappings (for UDF creation):")
print("\nRegion encoding:")
df_encoded.select("REGION", "REGION_ENCODED").distinct().sort("REGION_ENCODED").show()
print("\nCategory encoding:")
df_encoded.select("CATEGORY", "CATEGORY_ENCODED").distinct().sort("CATEGORY_ENCODED").show()

## 4. Train/Test Split

In [ ]:
# Define feature columns and target
# Note: UNITS_SOLD is now the TARGET, not a feature!
feature_cols = ["REGION_ENCODED", "CATEGORY_ENCODED", "MONTH", "DAY_OF_WEEK", "YEAR"]
target_col = "UNITS_SOLD"

# Split data 80/20
train_df, test_df = df_encoded.random_split([0.8, 0.2], seed=42)

print(f"Training records: {train_df.count()}")
print(f"Test records: {test_df.count()}")

## 5. Train XGBoost Model

In [ ]:
# Create and train XGBoost regressor to predict UNITS_SOLD
model = XGBRegressor(
    input_cols=feature_cols,
    label_cols=[target_col],
    output_cols=["PREDICTED_UNITS"],
    n_estimators=100,
    max_depth=6,
    learning_rate=0.1,
    random_state=42
)

print("Training XGBoost demand forecasting model...")
model.fit(train_df)
print("Training complete!")

## 6. Evaluate Model

In [ ]:
# Make predictions on test set
predictions_df = model.predict(test_df)

# Convert to pandas for evaluation
results_pd = predictions_df.select(target_col, "PREDICTED_UNITS").to_pandas()

# Calculate metrics
y_true = results_pd[target_col]
y_pred = results_pd["PREDICTED_UNITS"]

rmse = np.sqrt(mean_squared_error(y_true, y_pred))
mae = mean_absolute_error(y_true, y_pred)
r2 = r2_score(y_true, y_pred)

print("\n=== Model Performance ===")
print(f"RMSE: {rmse:,.1f} units")
print(f"MAE:  {mae:,.1f} units")
print(f"R2:   {r2:.3f}")

In [ ]:
# Show sample predictions
print("\nSample Predictions:")
predictions_df.select(
    "REGION", "CATEGORY", "MONTH", target_col, "PREDICTED_UNITS"
).show(10)

## 7. Register Model in Snowflake Model Registry

In [ ]:
# Initialize registry
registry = Registry(session=session, database_name=DATABASE, schema_name=SCHEMA)

# Log the model to registry - IMPORTANT: specify WAREHOUSE as target platform
model_version = registry.log_model(
    model_name=MODEL_NAME,
    version_name="v1",
    model=model,
    target_platforms=["WAREHOUSE"],  # Required for agent/SQL access
    comment=f"XGBoost demand forecasting model. Predicts units sold. RMSE: {rmse:,.1f} units, R2: {r2:.3f}"
)

print(f"Model registered: {MODEL_NAME} v1")
print(f"Full path: {DATABASE}.{SCHEMA}.{MODEL_NAME}")
print(f"Target platforms: WAREHOUSE (can be called via SQL/agent)")

In [ ]:
# List all models in registry
print("\nModels in registry:")
for m in registry.models():
    print(f"  - {m.name}")

## 8. Test Model Inference via SQL

In [ ]:
# Test the model with sample data
test_query = f"""
WITH sample_data AS (
    SELECT 
        1 AS REGION_ENCODED,  -- e.g., 'West'
        0 AS CATEGORY_ENCODED, -- e.g., 'Electronics'
        8 AS MONTH,
        3 AS DAY_OF_WEEK,
        2025 AS YEAR
)
SELECT * FROM sample_data
"""

sample_df = session.sql(test_query)
prediction = model.predict(sample_df)
prediction.show()

## 9. Call Model via SQL

The registered model can be called directly using SQL. This is the recommended approach for using ML models in Snowflake.

In [ ]:
# Call the registered model directly via SQL
# The model exposes a PREDICT method that can be called using: MODEL_NAME!PREDICT(...)

inference_sql = f"""
WITH sample_input AS (
    SELECT 
        1.0::FLOAT AS REGION_ENCODED,      -- West
        0.0::FLOAT AS CATEGORY_ENCODED,    -- Electronics
        8.0::FLOAT AS MONTH,
        3.0::FLOAT AS DAY_OF_WEEK,
        2025.0::FLOAT AS YEAR
)
SELECT {DATABASE}.{SCHEMA}.{MODEL_NAME}!PREDICT(
    REGION_ENCODED, CATEGORY_ENCODED, MONTH, DAY_OF_WEEK, YEAR
):PREDICTED_UNITS::FLOAT AS predicted_units
FROM sample_input
"""

result = session.sql(inference_sql).collect()
print(f"Predicted units sold (via SQL): {result[0]['PREDICTED_UNITS']:,.0f} units")

## 10. Create SQL UDF for Agent Integration

To make the model callable by a Snowflake Intelligence Agent, we create a SQL UDF that wraps the model prediction.
This UDF accepts human-readable parameters (region name, category name) and handles the encoding internally.

In [ ]:
# Create SQL UDF that wraps the model for agent use
# This UDF accepts human-readable inputs and returns predicted units sold

create_udf_sql = f"""
CREATE OR REPLACE FUNCTION {DATABASE}.{SCHEMA}.FORECAST_DEMAND(
    region VARCHAR,
    category VARCHAR,
    prediction_month INT
)
RETURNS FLOAT
COMMENT = 'Forecasts units sold (demand) for a given region, category, and month. Use this to predict how many units will sell.'
AS
$$
    SELECT {DATABASE}.{SCHEMA}.{MODEL_NAME}!PREDICT(
        CASE region 
            WHEN 'East' THEN 0 
            WHEN 'North' THEN 1 
            WHEN 'South' THEN 2 
            ELSE 3 
        END::FLOAT,
        CASE category 
            WHEN 'Electronics' THEN 0 
            WHEN 'Fitness Wear' THEN 1 
            WHEN 'Home Appliances' THEN 2 
            WHEN 'Outdoor Gear' THEN 3 
            ELSE 4 
        END::FLOAT,
        prediction_month::FLOAT,
        3::FLOAT,
        2025::FLOAT
    ):PREDICTED_UNITS::FLOAT
$$
"""

session.sql(create_udf_sql).collect()
print(f"Created UDF: {DATABASE}.{SCHEMA}.FORECAST_DEMAND")
print("\nUsage: SELECT FORECAST_DEMAND('West', 'Electronics', 8)")
print("Returns: Predicted number of units that will sell")

In [ ]:
# Test the UDF
test_udf_sql = f"""
SELECT 
    'West' AS region,
    'Electronics' AS category,
    8 AS month,
    ROUND({DATABASE}.{SCHEMA}.FORECAST_DEMAND('West', 'Electronics', 8)) AS predicted_units
"""

print("Testing FORECAST_DEMAND UDF:")
session.sql(test_udf_sql).show()

In [ ]:
# Test multiple forecasts
multi_test_sql = f"""
SELECT 
    region,
    category,
    month,
    ROUND({DATABASE}.{SCHEMA}.FORECAST_DEMAND(region, category, month)) AS forecasted_units
FROM (
    SELECT 'West' AS region, 'Electronics' AS category, 8 AS month
    UNION ALL
    SELECT 'East', 'Fitness Wear', 9
    UNION ALL
    SELECT 'North', 'Home Appliances', 12
    UNION ALL
    SELECT 'South', 'Outdoor Gear', 6
)
"""

print("Sample demand forecasts across regions and categories:")
session.sql(multi_test_sql).show()

## Done!

The demand forecasting model is now registered and a UDF is available for the Snowflake Intelligence Agent.

### What was created:

1. **Model**: `SI_DB.RETAIL.SI_DEMAND_FORECAST` - XGBoost regression model that predicts units sold
2. **UDF**: `SI_DB.RETAIL.FORECAST_DEMAND(region, category, month)` - Agent-callable function

### How to use the UDF:
```sql
-- Predict how many units of Electronics will sell in the West in August
SELECT FORECAST_DEMAND('West', 'Electronics', 8);

-- Valid regions: 'East', 'West', 'North', 'South'
-- Valid categories: 'Electronics', 'Fitness Wear', 'Home Appliances', 'Smart Home', 'Outdoor Gear'
-- Month: 1-12
```

### Add to Snowflake Intelligence Agent:

1. Go to **Snowflake Intelligence** in Snowsight
2. Edit your agent
3. Click **+ Add Tool** and select **Function**
4. Select `SI_DB.RETAIL.FORECAST_DEMAND`
5. The agent can now forecast demand! Try asking:
   - *"How many units of Electronics will we sell in the West next month?"*
   - *"What's the forecasted demand for Fitness Wear in Q4?"*
   - *"Compare expected sales volume between regions for Home Appliances"*